In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9800,2024-05-01,Espanha Acb,15:45,Tenerife,Unicaja,1.90,1.90,163.5,1.80,1.96,-1.5,2.01,2.03,MD51LWFp,0.526316,0.526316,0.555556,0.510204,0.052632,0.4,0.4,NaN,NaN,142.318,28.201898,0.198161,144.324,34.811432,0.241203,127.376,131.506,147.25,134.64,0.0,0.0,0.0,0.0,0.000000,0.060179,0.007001,-0.75,-0.150,-6.000000,0.000000,0.0,0.000000,1.17,0.234,3.846154,0.624232,0.9,0.275768
9801,2024-05-01,Eua Nba,23:00,Los Angeles Clippers,Dallas Mavericks,2.27,1.68,208.5,1.93,1.93,1.5,2.05,1.53,KKFEp2gM,0.440529,0.595238,0.518135,0.518135,0.035767,0.8,0.2,NaN,NaN,238.452,25.358738,0.106347,176.960,29.204304,0.165033,219.342,209.244,196.23,170.88,0.0,0.0,0.0,0.0,0.211237,0.000000,0.205416,-3.48,-0.696,-1.824713,0.623034,0.3,-0.323034,4.12,0.824,0.825243,0.555933,0.8,0.244067
9802,2024-05-01,Europa Euroliga,14:45,Fenerbahce,Monaco,1.51,2.64,165.5,1.91,1.91,-4.5,1.91,3.75,r3hgEoBj,0.662252,0.378788,0.523560,0.523560,0.041040,0.2,0.2,NaN,NaN,174.268,45.190578,0.259317,129.856,36.628032,0.282067,124.928,153.254,140.80,106.20,0.0,0.0,0.0,0.0,0.385075,0.000000,0.459744,1.66,0.332,1.536145,0.765001,0.9,0.134999,2.68,0.536,3.059701,0.536416,0.6,0.063584
9803,2024-05-01,Alemanha Bbl,10:30,Tubingen,Braunschweig,2.31,1.62,171.5,1.88,1.92,2.5,1.99,1.39,lG3751qm,0.432900,0.617284,0.531915,0.520833,0.050184,0.4,0.0,NaN,NaN,202.118,105.192636,0.520452,173.534,77.146313,0.444560,190.798,210.358,142.50,146.00,0.0,0.0,0.0,0.0,0.248297,0.014886,0.251044,-3.22,-0.644,-2.034161,0.421084,0.3,-0.121084,-1.46,-0.292,-2.123288,0.351579,0.3,-0.051579
9804,2024-05-01,Alemanha Bbl,10:30,Ulm,Bayern,2.29,1.63,166.5,1.82,1.96,2.5,1.97,1.40,4rSzBNLP,0.436681,0.613497,0.549451,0.510204,0.050178,0.4,0.6,NaN,NaN,203.826,70.082246,0.343834,149.772,39.492844,0.263686,105.914,189.326,102.09,139.50,0.0,0.0,0.0,0.0,0.238107,0.052378,0.239199,-1.53,-0.306,-4.215686,0.000000,0.0,0.000000,1.29,0.258,2.441860,0.484354,0.5,0.015646
9805,2024-05-01,Alemanha Dbbl Feminina,10:30,Alba Berlin F,Keltern F,1.78,1.94,122.5,1.84,1.86,-1.5,1.90,2.06,ENjUPQAM,0.561798,0.515464,0.543478,0.537634,0.077262,0.6,0.4,NaN,NaN,106.316,23.513870,0.221170,88.090,13.541507,0.153724,100.084,90.768,69.76,93.96,0.0,0.0,1.0,0.0,0.060826,0.007644,0.057140,2.22,0.444,1.756757,0.000000,0.0,0.000000,0.66,0.132,7.121212,0.000000,0.0,0.000000
9806,2024-05-01,Bósnia E Herzegovina Prvenstvo Bih,14:30,Posusje,Leotar Trebinje,2.15,1.65,167.5,1.85,1.85,1.5,2.00,1.51,M1sDsK0j,0.465116,0.606061,0.540541,0.540541,0.071177,0.6,0.8,NaN,NaN,215.816,73.632475,0.341182,214.404,155.602728,0.725745,243.432,294.564,199.95,143.55,0.0,0.0,0.0,0.0,0.186081,0.000000,0.197426,-1.09,-0.218,-5.275229,0.000000,0.0,0.000000,-1.63,-0.326,-1.993865,0.000000,0.0,0.000000
9807,2024-05-01,Estônia Korvpalli Meistriliiga,13:30,Keila,Pärnu,1.95,1.77,152.5,1.85,1.85,1.5,1.82,1.62,baQIMuKl,0.512821,0.564972,0.540541,0.540541,0.077792,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.068430,0.000000,0.082222,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9808,2024-05-01,Europa Liga Bnxt,14:30,Zwolle,Liege,1.98,1.75,157.5,1.85,1.85,1.5,1.85,1.61,QmosARnE,0.505051,0.571429,0.540541,0.540541,0.076479,0.4,1.0,NaN,NaN,104.212,17.933341,0.172085,140.322,21.439746,0.152790,109.712,275.038,129.28,150.75,0.0,0.0,0.0,0.0,0.087204,0.000000,0.098096,-1.20,-0.240,-4.083333,0.000000,0.0,0.000000,3.02,0.604,1.241722,0.000000,0.0,0.000000
980

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,15:45,Espanha Acb,Tenerife,Unicaja,163.5,1.80,0.9998,Over
1,14:45,Europa Euroliga,Fenerbahce,Monaco,165.5,1.91,1.0000,Over
2,10:30,Alemanha Dbbl Feminina,Alba Berlin F,Keltern F,122.5,1.84,0.9984,Over
3,13:00,Hungria Nacional Feminino,Szekszard F,Sopron F,139.5,1.87,1.0000,Over
4,14:45,Israel Superliga,Hapoel Beer Sheva,Ironi Kiryat Ata,171.5,1.86,1.0000,Over
